In [1]:
from simhash import Simhash
from collections import defaultdict

# Hàm tính SimHash cho văn bản
def get_simhash(text):
    return Simhash(str(text), f=128).value

# Hàm tính khoảng cách Hamming giữa hai SimHash
def hamming_distance(hash1, hash2):
    x = hash1 ^ hash2
    dist = bin(x).count('1')
    return dist

# Áp dụng LSH để nhóm các SimHash tương tự lại với nhau và giữ lại 1 văn bản đại diện cho mỗi nhóm
def lsh_grouping(texts, threshold, num_bands):
    simhashes = [(i, get_simhash(text)) for i, text in enumerate(texts)]
    buckets = defaultdict(list)

    # Chia SimHash thành các bands
    for idx, simhash in simhashes:
        hash_str = f'{simhash:128b}'  # Chuyển SimHash thành chuỗi nhị phân 128 bit
        band_size = len(hash_str) // num_bands
        for band in range(num_bands):
            band_hash = hash_str[band * band_size:(band + 1) * band_size]
            buckets[(band, band_hash)].append(idx)

    # Sử dụng DFS để gom các nhóm dữ liệu có Hamming distance <= threshold
    visited = set()
    unique_groups = []

    def dfs(index, group):
        visited.add(index)
        group.append(index)
        for neighbor in neighbors[index]:
            if neighbor not in visited:
                dfs(neighbor, group)

    # Xây dựng danh sách các hàng xóm dựa trên khoảng cách Hamming
    neighbors = defaultdict(list)
    for bucket, indices in buckets.items():
        for i in range(len(indices)):
            for j in range(i + 1, len(indices)):
                idx1, idx2 = indices[i], indices[j]
                hash1, hash2 = simhashes[idx1][1], simhashes[idx2][1]
                if hamming_distance(hash1, hash2) <= threshold:
                    neighbors[idx1].append(idx2)
                    neighbors[idx2].append(idx1)

    # Gom các nhóm dựa trên DFS và chỉ giữ lại 1 đại diện cho mỗi nhóm
    output_group = []
    for idx in range(len(texts)):
        if idx not in visited:
            group = []
            dfs(idx, group)
            unique_groups.append(group[0])  # Giữ lại chỉ một đại diện trong mỗi nhóm

    # Trả về các văn bản không bị trùng lặp
    return [texts[i] for i in unique_groups]


In [2]:
import re

# Hàm loại bỏ mã code
def remove_codes(text):
    # Mẫu regex để tìm các chuỗi mã có chữ cái và số (ít nhất 5 ký tự)
    pattern = r'\b\w{5,}\b'
    
    # Loại bỏ các chuỗi mã code
    cleaned_text = re.sub(pattern, '', text)
    
    # Loại bỏ các khoảng trắng dư thừa
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

def remove_duplicates(array):
    seen = set()  # Tạo một set để lưu các chuỗi đã gặp
    unique_array = []
    
    for i in range(len(array)):
        if array[i] not in seen:
            unique_array.append(i)
            seen.add(array[i])
    
    return unique_array

In [3]:
import pandas as pd

texts = pd.read_csv('data_duplicate.csv')['text']
new_texts = []
for text in texts:
    new_texts.append(remove_codes(str(text)))

In [ ]:
import time

start_time = time.time()
result = remove_duplicates(new_texts)
unique_texts = [texts[i] for i in result]

output_texts = lsh_grouping(texts=unique_texts, threshold=20, num_bands=16)
# unique_texts = [texts[i] for i in indices]

end_time = time.time()

# Tính thời gian chạy
execution_time = end_time - start_time
print(f"Thời gian chạy: {execution_time:.4f} giây")

Thời gian chạy: 1.8745 giây


In [7]:
len(unique_texts)

5383

In [8]:
len(output_texts)

4717

In [9]:
output = pd.DataFrame({'text':output_texts})
output.to_csv('new_data_simhash.csv')